# Model Specific Data Prep

### Analyzing columns to select which one to use as target/label\

NOTE: I did this by accident. In trying to make the lead scoring model, I accidentally created the lead scoring model.

Here, I ended up choosing won_deal. The classifier should ultimately come up with a probability that won_deal is either 0 or 1 and classify based on that.

In [107]:
import pandas as pd

df = pd.read_csv("../data/processed/cleaned_data.csv")

# Look for deal-related columns
cols = [c for c in df.columns if "deal" in c.lower() or "stage" in c.lower() or "won" in c.lower() or "close" in c.lower()]
print(cols)

# Peek at their unique values
for c in cols:
    print(f"\nColumn: {c}")
    print(df[c].value_counts(dropna=False).head(10))


['close_date', 'close_value', 'has_close_date', 'close_value_log', 'deal_stage_ENGAGING', 'deal_stage_LOST', 'deal_stage_PROSPECTING', 'deal_stage_WON', 'days_to_close', 'closed_within_30d', 'agent_closed_deals', 'won_deal']

Column: close_date
close_date
NaN           2089
2017-05-22      41
2017-06-11      36
2017-11-19      34
2017-05-20      34
2017-08-11      34
2017-05-26      34
2017-11-07      33
2017-11-03      33
2017-05-06      33
Name: count, dtype: int64

Column: close_value
close_value
0.0     2473
NaN     2089
54.0      80
53.0      57
57.0      55
59.0      54
55.0      52
56.0      48
51.0      45
60.0      44
Name: count, dtype: int64

Column: has_close_date
has_close_date
1    6711
0    2089
Name: count, dtype: int64

Column: close_value_log
close_value_log
0.000000    2473
NaN         2089
4.007333      80
3.988984      57
4.060443      55
4.094345      54
4.025352      52
4.043051      48
3.951244      45
4.110874      44
Name: count, dtype: int64

Column: deal_sta

I realized theres probably a lot of columns that would help the model "cheat" in figuring out if the deal closes as a win or not. Here, I'm trying to drop all columns that do that.

In [108]:
df.columns

Index(['sales_agent', 'account', 'engage_date', 'close_date', 'close_value',
       'sales_price', 'year_established', 'revenue', 'employees',
       'office_location', 'has_close_date', 'close_value_log', 'revenue_log',
       'employees_log', 'product_GTK 500', 'product_GTX BASIC',
       'product_GTX PLUS BASIC', 'product_GTX PLUS PRO', 'product_GTX PRO',
       'product_MG ADVANCED', 'product_MG SPECIAL', 'manager_CARA LOSCH',
       'manager_CELIA ROUCHE', 'manager_DUSTIN BRINKMANN',
       'manager_MELVIN MARXEN', 'manager_ROCCO NEUBERT',
       'manager_SUMMER SEWALD', 'regional_office_CENTRAL',
       'regional_office_EAST', 'regional_office_WEST', 'series_GTK',
       'series_GTX', 'series_MG', 'sector_EMPLOYMENT', 'sector_ENTERTAINMENT',
       'sector_FINANCE', 'sector_MARKETING', 'sector_MEDICAL', 'sector_RETAIL',
       'sector_SERVICES', 'sector_SOFTWARE', 'sector_TECHNOLGY',
       'sector_TELECOMMUNICATIONS', 'subsidiary_of_ACME CORPORATION',
       'subsidiary_of_BUBBA

In [109]:
label = "won_deal"

leak_cols = ["deal_stage_WON", "deal_stage_LOST", "deal_stage_ENGAGING", "deal_stage_PROSPECTING", "close_date", "close_value", "has_close_date", "close_value_log", "days_to_close", "closed_within_30d", "agent_closed_deals"]

df = df.drop(columns=[col for col in leak_cols if col in df.columns])

df.columns

Index(['sales_agent', 'account', 'engage_date', 'sales_price',
       'year_established', 'revenue', 'employees', 'office_location',
       'revenue_log', 'employees_log', 'product_GTK 500', 'product_GTX BASIC',
       'product_GTX PLUS BASIC', 'product_GTX PLUS PRO', 'product_GTX PRO',
       'product_MG ADVANCED', 'product_MG SPECIAL', 'manager_CARA LOSCH',
       'manager_CELIA ROUCHE', 'manager_DUSTIN BRINKMANN',
       'manager_MELVIN MARXEN', 'manager_ROCCO NEUBERT',
       'manager_SUMMER SEWALD', 'regional_office_CENTRAL',
       'regional_office_EAST', 'regional_office_WEST', 'series_GTK',
       'series_GTX', 'series_MG', 'sector_EMPLOYMENT', 'sector_ENTERTAINMENT',
       'sector_FINANCE', 'sector_MARKETING', 'sector_MEDICAL', 'sector_RETAIL',
       'sector_SERVICES', 'sector_SOFTWARE', 'sector_TECHNOLGY',
       'sector_TELECOMMUNICATIONS', 'subsidiary_of_ACME CORPORATION',
       'subsidiary_of_BUBBA GUMP', 'subsidiary_of_GOLDDEX',
       'subsidiary_of_INITY', 'subsidiar

In [110]:
# finding all non-numeric columns leftover
non_numeric = df.select_dtypes(exclude=['number']).columns
print(non_numeric)

Index(['engage_date'], dtype='object')


In [111]:
df.drop("engage_date", axis=1) # dropping because its still not numerical (can't be used by model) and thus far we haven't been able to find any relevancy

,sales_agent,account,sales_price,year_established,revenue,employees,office_location,revenue_log,employees_log,product_GTK 500,...,decade_1990s,decade_2000s,decade_2010s,engage_year,engage_month,engage_dayofweek,account_age,rev_per_employee,won_deal,account_win_rate
0,20,8,1096,2001.0,718.62,2448.0,14,6.578723,7.803435,0,...,0,1,0,2016.0,10.0,3.0,24.0,0.293554,1,0.920792
1,6,38,4821,2002.0,3178.24,4540.0,14,8.064397,8.420903,0,...,0,1,0,2016.0,10.0,1.0,23.0,0.700053,1,0.941176
2,6,8,55,2001.0,718.62,2448.0,14,6.578723,7.803435,0,...,0,1,0,2016.0,10.0,1.0,24.0,0.293554,1,0.920792
3,20,10,550,1998.0,2714.90,2641.0,14,7.906879,7.879291,0,...,1,0,0,2016.0,10.0,1.0,27.0,1.027982,1,0.966942
4,29,34,550,1982.0,792.46,1299.0,14,6.676403,7.170120,0,...,0,0,0,2016.0,10.0,1.0,43.0,0.610054,1,0.873563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,25,35,3393,1995.0,1698.20,3492.0,14,7.437913,8.158516,0,...,1,0,0,NaN,NaN,NaN,30.0,0.486312,0,0.118769
8796,25,35,3393,1995.0,1698.20,3492.0,14,7.437913,8.158516,0,...,1,0,0,NaN,NaN,NaN,30.0,0.486312,0,0.118769
8797,25,35,3393,1995.0,1698.20,3492.0,14,7.437913,8.158516,0,...,1,0,0,NaN,NaN,NaN,30.0,0.486312,0,0.118769
8798,25,35,3393,1995.0,1698.20,3492.0,14,7.437913,8.158516,0,...,1,0,0,NaN,NaN,NaN,30.0,0.486312,0,0.118769


# Begin Training the Model

In [112]:
!pip install xgboost

In [113]:
import xgboost
print(xgboost.__version__)


3.0.5


In [114]:
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [115]:
# X = df.drop(columns=[label])
# y = df[label]

# CLEAN UP LATER TO FIGURE OUT WHY NON NUMERICALS WERE COMING THROUGH

y = df[label].astype(int)          # target
X = df.drop(columns=[label])       # features
X = X.select_dtypes(include='number')         # keep only numeric features

from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')
X = imp.fit_transform(X)


In [116]:
# Split training set
# Potential TODO: Go back and do K-fold validation. Figure out how to do test, training, and validation sets again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # stratify preserves class proportions

# Models and baseline hyperparameters to be used for model selection:
models = {
    "RandomForest": RandomForestClassifier(
        n_estimators = 400,
        max_depth = 12,
        min_samples_split = 5,
        n_jobs = -1,
        random_state = 42
    ),
    "XGBoost": XGBClassifier(
        n_estimators = 600,
        max_depth = 6,
        learning_rate = 0.05,
        subsample = 0.9,
        colsample_bytree = 0.9,
        reg_lambda = 1.0,
        objective = "binary:logistic",
        eval_metric = "logloss",
        n_jobs = -1,
        random_state = 42
    ),
    "LogReg_L2": LogisticRegression(
        max_iter=2000,
        n_jobs = -1,
        C = 0.5
    )
}

# Helper function to evaluate each model
def evaluate(name, model):
    
    print("Fitting model:", name)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        from sklearn.preprocessing import MinMaxScaler
        y_proba = MinMaxScaler().fit_transform(model.decision_function(X_test).reshape(-1,1)).ravel()
    else:
        y_proba = y_pred  # fallback (not ideal)

    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary", zero_division=0)
    auc = roc_auc_score(y_test, y_proba)

    print(f"\n=== {name} ===")
    print(f"Accuracy:  {acc:.3f}")
    print(f"Precision: {prec:.3f}   Recall: {rec:.3f}   F1: {f1:.3f}   ROC-AUC: {auc:.3f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, digits=3))

    return acc
    

In [117]:
results = {name: evaluate(name, m) for name, m in models.items()}

best_model_name = max(results, key=results.get)
print(f"\nBest model by Accuracy: {best_model_name} ({results[best_model_name]:.3f})")

Fitting model: RandomForest

=== RandomForest ===
Accuracy:  0.947
Precision: 0.935   Recall: 1.000   F1: 0.966   ROC-AUC: 0.972

Confusion Matrix:
[[ 324   94]
 [   0 1342]]

Classification Report:
              precision    recall  f1-score   support

           0      1.000     0.775     0.873       418
           1      0.935     1.000     0.966      1342

    accuracy                          0.947      1760
   macro avg      0.967     0.888     0.920      1760
weighted avg      0.950     0.947     0.944      1760

Fitting model: XGBoost

=== XGBoost ===
Accuracy:  0.950
Precision: 0.949   Recall: 0.987   F1: 0.968   ROC-AUC: 0.969

Confusion Matrix:
[[ 347   71]
 [  17 1325]]

Classification Report:
              precision    recall  f1-score   support

           0      0.953     0.830     0.887       418
           1      0.949     0.987     0.968      1342

    accuracy                          0.950      1760
   macro avg      0.951     0.909     0.928      1760
weighted avg 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Further To Do:
1. Figure out why missing values/non numeric columns were slipping through to model testing
2. Figure out if model is overfitting. Consider: K-fold Cross Validation, Out of Sample Validation
3. Hyperparameter tuning
4. Very low priority: Experiment with Neural Networks? Hypothetically, it's not the best approach to this problem, but I'm curious to see how it compares and why
5. Research Gradio, mock presentation